# 10th Day of Python 4 DIP

> Contents:
> 1. Non local Maxima Supression
> 2. Canny and Hough edge detection
 

Execute the cells to gain outputs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage as SK
import time
from Tools import *
%matplotlib inline

### The Magnitude and directions 

In [ ]:
def Grad(Image):
    """
    Simple Gradient magnitude and angles calculator
    """
    Dx = SK.filters.sobel_h(Image)
    Dy = SK.filters.sobel_v(Image)
    M = np.sqrt(Dx**2+Dy**2)
    th = np.rad2deg(np.arctan2(Dy,Dx))
    th = (th + 360)%360
    return M,th

### The Non local Maxima Supression (NLS)

In [ ]:
def NLS(Magnitude,Theta):
    """The Simple Non local supression function"""
    M,N = Magnitude.shape
    result = np.zeros_like(Magnitude)

    ## Splitting the angles to 45 degree lines
    Theta = (np.floor((Theta+22.5)/45)*45)%360
    for i in range(1,M-1):
        for j in range(1,N-1):
            alpha = Theta[i,j]

            if alpha == 0.0 or alpha == 180.0:
                Indexs = [Magnitude[i,j-1],Magnitude[i,j+1]]
            elif alpha == 45.0 or alpha == 225.0:
                Indexs = [Magnitude[i+1,j+1],Magnitude[i-1,j-1]]
            elif alpha == 90.0 or alpha == 270.0:
                Indexs = [Magnitude[i-1,j],Magnitude[i+1,j]]
            elif alpha == 135.0 or alpha == 315.0:
                Indexs = [Magnitude[i+1,j],Magnitude[i-1,j+1]]
            else:
                raise ValueError(f"The alpha = {alpha} is not in [0,45,90,135,180,225,270,315].")
            
            if Magnitude[i,j] >= np.max(Indexs):
                result[i,j] = Magnitude[i,j]
            else:
                result[i,j] = 0.0
            
    return result


In [ ]:
Camera = SK.io.imread("./Figures/cameraman.jpg",as_gray=True)
M,Th = Grad(Camera)
Camera_NLS = NLS(M,Th)
ShowTimeMul((M*255,Camera_NLS*255),With_Hist=False,figsize=(7,5),Im_title=["Magnitude","NLS of Magnitude"],cmaps=['gray']*2)

### Strong and Weak edges

In [ ]:
def double_thresholding(Image,High,Low):
    """
    Simple Double thresholding function
    """
    Strong = np.zeros_like(Image)
    Weak = Strong.copy()
    Strong = Image > High
    Weak = (Image > Low) & (Image <= High)
    return Strong,Weak

In [ ]:
low = 0.35
high = 0.5
Strong,Weak = double_thresholding(Camera_NLS,high,low)
ShowTimeMul((Weak*255,Strong*255),With_Hist=False,figsize=(7,5),Im_title=["Weak","Strong"],cmaps=['gray']*2)

### Canny Edge detection with scikit-image

In [ ]:
Camera_Noised = SK.util.random_noise(Camera,mode="s&p",seed=1000,salt_vs_pepper = 0.2)
Camera_Smooth = SK.filters.gaussian(Camera_Noised,sigma=2)
Edges = SK.feature.canny(Camera_Smooth)
Edges2 = SK.feature.canny(Camera_Smooth,sigma=2)
ShowTimeMul((Edges,Edges2),With_Hist=False,figsize=(7,5),Im_title=["Edges","Edges 2"],cmaps=['gray']*2,colorbar=False)

### Robert and Sobel Edge detection

In [ ]:
Edges_Robert = SK.filters.roberts(Camera_Smooth)
Edges_Sobel = SK.filters.sobel(Camera_Smooth)

ShowTimeMul((Edges_Robert,Edges_Sobel),With_Hist=False,figsize=(7,5),Im_title=["Robert","Sobel"],cmaps=['gray']*2,colorbar=False)

### Hough transform

In [ ]:
Lines = SK.transform.probabilistic_hough_line(Edges,threshold=10,line_length=5,line_gap=3)
#ShowTime(Edges,figsize=(7,5),With_Hist=False,colorbar=False)
fig,ax = plt.subplots(nrows=1,ncols=1)
ax.imshow(Edges,cmap='gray')
for l in Lines:
    p0,p1 = l
    ax.plot((p0[0],p1[0]),(p0[1],p1[1]))

    